# MWE of PCH: Periodic deblurring problem

First import packages required

In [ ]:
import sys

sys.path.append("..") 
import numpy as np
from scipy.sparse import diags
import matplotlib.pyplot as plt
import inspect

# Set rng seed 
np.random.seed(0)

Then import tools needed from CUQIpy:

In [ ]:
from cuqi.testproblem import Deconvolution1D
from cuqi.model import LinearModel
from cuqi.distribution import Gaussian, LMRF, CMRF
from cuqi.sampler import CWMH
from cuqi.problem import BayesianProblem
from cuqi.samples import Samples

## 1. Loading the Deconvolution problem

We want to experiment with the Deconvolution test problem; we first take a look at how to call it:

In [ ]:
help(Deconvolution1D)

Specify parameters and create Deconvolution TestProblem:

In [ ]:
dim = 128
kernel = ["Gauss","Sinc","vonMises"]
phantom = ["Gauss","Sinc","vonMises","Square","Hat","Bumps","DerivGauss"]
noise_type = ["Gaussian","ScaledGaussian"]
noise_std = 0.05

# Test problem
prob = Deconvolution1D(
    dim = dim,
    kernel=kernel[0],
    phantom=phantom[3],
    noise_type=noise_type[0],
    noise_std = noise_std
)

A CUQI TestProblem already contains everything apart from a prior to do UQ, so we could simply specify a prior and then run the UQ method directly on the test problem. This is because TestProblem is a subclass of BayesianProblem, so the UQ method is inherited. But, here we will "unpack" the main ingredients (the linear operator and the original clean signal) and build our problem from scratch.

In [ ]:
A = prob.model.get_matrix()
phantom = prob.exactSolution     # We may later return and generate phantom as a sample from a prior distribution.

We can show the matrix and the "original clean signal" directly using python matplotlib plotting methods:

In [ ]:
plt.figure()
plt.imshow(A)
plt.figure()
plt.plot(phantom)

## 2. Set up problem in CUQIpy with IID Gaussian prior

We first set up a CUQI LinearModel:

In [ ]:
# Define as linear model
model = LinearModel(A)

With the CUQI model we can for example apply it to the clean signal and obtain clean data and plot it:

In [ ]:
data_clean = model(phantom)
plt.plot(data_clean)

We can use the geometries of the domain and range of the model for the dedicated plot type (for a simple 1D signal like this, the default Continuous1D geometry is automatically employed and its plotting method simply add the xlabel)

In [ ]:
model.domain_geometry.plot(phantom)
plt.figure()
model.range_geometry.plot(data_clean)

Define data distribution as Gaussian with model as mean and i.i.d. noise with 0.05 std.  Note how the CUQIpy object "model" is given as the mean. Thus data distribution is a conditional distribution, which can only be sampled once the conditioning variable is specified.

In [ ]:
noise_std = 0.05
data_dist = Gaussian(model, noise_std, np.eye(dim))

Generate and plot samples of noise by conditioning on the zero image:

In [ ]:
data_dist(np.zeros(dim)).sample(5).plot()
plt.title('Noise samples'); plt.show()

In the same way, by conditioning on the true phantom, we generate (noisy) samples of simulated data:

In [ ]:
data_dist(phantom).sample(5).plot()
plt.title('Simulated data'); plt.show()

Now generate single realization of data to be used in inverse problem:

In [ ]:
data = data_dist(phantom).sample()

Specify the prior distribution and generate and plot 5 samples:

In [ ]:
prior_std = 0.2
prior = Gaussian(np.zeros(dim), prior_std, np.eye(dim), name="x") # Set name to "x" to match model input

# Plot samples of prior
prior.sample(5).plot()
plt.title('Realizations from prior'); plt.show()

Combine likelihood, prior and the observed data into "BayesianProblem" object representing the inverse problem. No solving yet.

In [ ]:
likelihood = data_dist.to_likelihood(data)

IP = BayesianProblem(likelihood, prior)

## 3. Solving the inverse problem

We can ask for the MAP estimate:  (for small fully Gaussian problems we use analytical expression; for more complicated problems support is underway using numerical optimization. Prototype support implemented at hackathon using gradients of logpdf and LBFGS from scipy)

In [ ]:
x_MAP = IP.MAP() 

# Plot
plt.plot(phantom,'.-')
plt.plot(x_MAP,'.-')
plt.title("Map estimate")
plt.legend(["Exact","MAP"])
plt.show()

Instead of MAP, we can sample the posterior:

In [ ]:
Ns = 5000   # Number of samples
result = IP.sample_posterior(Ns)

The output is a cuqi.samples.Samples object containing the samples and various utilities such as plotting methods:

In [ ]:
type(result)

For example we can plot credibility interval around the exact:

In [ ]:
result.plot_ci(95, exact=phantom)

And the posterior sample std:

In [ ]:
result.plot_std()

As well as chains of the individual parameters, here three shown:

In [ ]:
idx = [20,55,60]
result.plot_chain(idx)
plt.legend(idx)

Diagnostics of the sampling is ongoing work, some information is available currently in result:

In [ ]:
result.diagnostics()

## 4. Change to a Gaussian with correlation

Define correlation matrix where 30 closest neighbours are correlated

In [ ]:
l = 30
corr = np.linspace(0,1,int(l/2)+1)
corr = np.hstack((corr,np.flipud(corr[:-1])))
indexes = np.linspace(-l/2,l/2,l+1,dtype=int)
corrmat = diags(corr, indexes, shape=(dim,dim)).toarray()

plt.imshow(corrmat)

Set new prior and plot a few samples:

In [ ]:
IP.prior = Gaussian(np.zeros(dim), prior_std, corrmat, name="x")

# Plot samples from prior
IP.prior.sample(5).plot() 
plt.title('Realizations from prior'); plt.show()

We have already updated the "prior" attribute of the IP, so we are ready to solve for the MAP and sample:

In [ ]:
x_MAP_corr = IP.MAP()

# Plot
plt.plot(phantom,'.-')
plt.plot(x_MAP_corr,'.-')
plt.title("Map estimate")
plt.legend(["Exact","MAP"])
plt.show()

In [ ]:
result_corr = IP.sample_posterior(Ns)

# plot mean + 95% of samples
result_corr.plot_ci(95, exact=phantom)

## 5. Change to a Cauchy prior

The correlated Gaussian gives a narrow credibility interval so was better than the IID Gaussian, but still cannot reproduce the sharp edges. We try a Cauchy prior.

In [ ]:
scale = 2/dim
IP.prior = CMRF(np.zeros(dim), scale, 'neumann', name="x")

MAP for Cauchy.

In [ ]:
x_MAP_cau = IP.MAP()

# Plot
plt.plot(phantom,'.-')
plt.plot(x_MAP_cau[0],'.-')
plt.title("Map estimate")
plt.legend(["Exact","MAP"])
plt.show()

Sampling of posterior for Cauchy. Different sampling method than Gaussian and this is slower.

In [ ]:
result_cau = IP.sample_posterior(Ns//10) #We dont need as many samples when the sampler is NUTS.

Plot mean + 95% of samples

In [ ]:
result_cau.plot_ci(95, exact=phantom)

In [ ]:
result_cau.plot_std()

## 6. Change to Laplace prior

Cauchy is known to be very edge-preserving but difficult to work with for example sample. Another edge-preserving prior is Laplace which typically is more well-behaved but not quite as strongly edge-preserving

In [ ]:
# Set up Laplace prior
location = 0
delta = 0.5
scale = delta*1/dim
IP.prior = LMRF(location, scale, 'zero', geometry=dim, name="x")

This runs yet another sampler

In [ ]:
result_lap = IP.sample_posterior(Ns//10)

Then we can use the plotting methods such as of the credibility interval:

In [ ]:
result_lap.plot_ci(95, exact=phantom)

Or std plot:

In [ ]:
result_lap.plot_std()

## Some ideas to try out (warning: not tested!)
1. Change phantom to another of the options provided by the TestProblem.
2. Change phantom to be a sample from Gaussian, then do UQ with correct and incorrect choices of prior.
3. Play with parameter of priors for example Cauchy and Laplace to see if a better solution can be obtained.
3. Change noise/likelihood to scaledGaussian